In [29]:
import pandas as pd
import numpy as np

In [2]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df1 = pd.read_csv('SML-dataset/weather_daily_darksky.csv', delimiter=',')

In [3]:
df1['summary'].value_counts()

Mostly cloudy throughout the day.                                  174
Partly cloudy throughout the day.                                  170
Partly cloudy until evening.                                       133
Mostly cloudy until evening.                                       118
Foggy in the morning.                                               47
                                                                  ... 
Overcast throughout the day and breezy overnight.                    1
Partly cloudy until evening and breezy starting in the evening.      1
Mostly cloudy until evening and breezy in the evening.               1
Breezy in the morning and overcast until evening.                    1
Windy and mostly cloudy until afternoon.                             1
Name: summary, Length: 88, dtype: int64

In [4]:
df2= pd.read_csv('/Users/dguiraldes/Documents/ISCI/intro_smart_meters/SML-dataset/halfhourly_dataset/halfhourly_dataset/block_0.csv')

In [5]:
df2[(df2['LCLid']=='MAC000002') & (df2['tstp']>='2013-01-01') & (df2['tstp']<'2014-01-01')]

,LCLid,tstp,energy(kWh/hh)
3837,MAC000002,2013-01-01 00:00:00.0000000,0.219
3838,MAC000002,2013-01-01 00:30:00.0000000,0.241
3839,MAC000002,2013-01-01 01:00:00.0000000,0.191
3840,MAC000002,2013-01-01 01:30:00.0000000,0.235
3841,MAC000002,2013-01-01 02:00:00.0000000,0.182
...,...,...,...
21352,MAC000002,2013-12-31 21:30:00.0000000,0.451
21353,MAC000002,2013-12-31 22:00:00.0000000,0.456
21354,MAC000002,2013-12-31 22:30:00.0000000,0.402
21355,MAC000002,2013-12-31 23:00:00.0000000,0.491


In [6]:
df_temps=df1[['time','temperatureMax','temperatureLow']]
df_temps['time']=pd.to_datetime(df_temps['time']).dt.date
df_temps.rename(columns={'time':'day'},inplace=True)

/var/folders/vt/5wzq72vd2mj_qqq_fwp77h_40000gq/T/ipykernel_13806/3102040786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps['time']=pd.to_datetime(df_temps['time']).dt.date
/var/folders/vt/5wzq72vd2mj_qqq_fwp77h_40000gq/T/ipykernel_13806/3102040786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps.rename(columns={'time':'day'},inplace=True)


In [7]:
df3=pd.read_csv('SML-dataset/daily_dataset.csv')

In [8]:
df_de=df3[df3['energy_count']==48].groupby('day').agg({'energy_sum':'mean'}).reset_index()
df_de['day']=pd.to_datetime(df_de['day']).dt.date

In [19]:
df_plot=df_temps.merge(df_de, on ='day').sort_values(by='day')

In [31]:
import seaborn as sns
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

In [26]:
fig= px.line(df_plot, x='day', y='energy_sum')

fig.update_xaxes(rangeslider_visible=True)

In [21]:
fig.show()

In [25]:
df_plot.corr()

,temperatureMax,temperatureLow,energy_sum
temperatureMax,1.000000,0.882333,-0.872188
temperatureLow,0.882333,1.000000,-0.793413
energy_sum,-0.872188,-0.793413,1.000000


In [34]:

df_plot2=df_plot.set_index("day")

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(df_plot2, y=['energy_sum'], render_mode="webgl",)
fig2 = px.line(df_plot2, y=['temperatureMax','temperatureLow'], render_mode="webgl",)

fig2.update_traces(yaxis="y2")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="day"
subfig.layout.yaxis.title="Avg daily energy (kwh)"
subfig.layout.yaxis2.title="Temperature (ºC)"


subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.update_xaxes(rangeslider_visible=True)
subfig.show()